In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect('sqlite_db_pythonsqlite.db')
cur = con.cursor()

In [3]:
pd.read_sql_query("SELECT * FROM sqlite_schema WHERE type='table'", con)

,type,name,tbl_name,rootpage,sql
0,table,Bookings,Bookings,2,"CREATE TABLE ""Bookings"" (\n ""bookid"" int(4) N..."
1,table,Facilities,Facilities,50,"CREATE TABLE ""Facilities"" (\n ""facid"" int(1) ..."
2,table,Members,Members,52,"CREATE TABLE ""Members"" (\n ""memid"" int(2) NOT..."


In [4]:
pd.read_sql_query('PRAGMA table_info(bookings)', con)

,cid,name,type,notnull,dflt_value,pk
0,0,bookid,int(4),1,'0',1
1,1,facid,int(1),0,NULL,0
2,2,memid,int(2),0,NULL,0
3,3,starttime,varchar(19),0,NULL,0
4,4,slots,int(2),0,NULL,0


In [5]:
pd.read_sql_query('PRAGMA table_info(facilities)', con)

,cid,name,type,notnull,dflt_value,pk
0,0,facid,int(1),1,'0',1
1,1,name,varchar(15),0,NULL,0
2,2,membercost,"decimal(2,1)",0,NULL,0
3,3,guestcost,"decimal(3,1)",0,NULL,0
4,4,initialoutlay,int(5),0,NULL,0
5,5,monthlymaintenance,int(4),0,NULL,0


In [6]:
pd.read_sql_query('PRAGMA table_info(members)', con)

,cid,name,type,notnull,dflt_value,pk
0,0,memid,int(2),1,'0',1
1,1,surname,varchar(17),0,NULL,0
2,2,firstname,varchar(9),0,NULL,0
3,3,address,varchar(39),0,NULL,0
4,4,zipcode,int(5),0,NULL,0
5,5,telephone,varchar(14),0,NULL,0
6,6,recommendedby,varchar(2),0,NULL,0
7,7,joindate,varchar(19),0,NULL,0


### Question 1: 
Some of the facilities charge a fee to members, but some do not. Write a SQL query to produce a list of the names of the facilities that do.

In [12]:
pd.read_sql_query('SELECT DISTINCT membercost FROM facilities LIMIT 5', con)

,membercost
0,5.0
1,0.0
2,9.9
3,3.5


In [11]:
query = '''
SELECT name
FROM facilities
WHERE membercost=0.0
'''
pd.read_sql_query(query, con)

,name
0,Badminton Court
1,Table Tennis
2,Snooker Table
3,Pool Table


## Question 2:
How many facilities do not charge a fee to members?

In [14]:
query = '''
SELECT COUNT(DISTINCT name) AS num_fac_zero_mem_cost
FROM facilities
WHERE membercost=0.0
'''
pd.read_sql_query(query, con)

,num_fac_zero_mem_cost
0,4


## Question 3:
Write an SQL query to show a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost. Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [15]:
query = '''
SELECT facid, name, membercost, monthlymaintenance
FROM facilities
WHERE membercost < 0.2*monthlymaintenance
'''
pd.read_sql_query(query, con)

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,2,Badminton Court,0.0,50
3,3,Table Tennis,0.0,10
4,4,Massage Room 1,9.9,3000
5,5,Massage Room 2,9.9,3000
6,6,Squash Court,3.5,80
7,7,Snooker Table,0.0,15
8,8,Pool Table,0.0,15


## Question 4:
Write an SQL query to retrieve the details of facilities with ID 1 and 5. Try writing the query without using the OR operator.

In [17]:
query = '''
SELECT * 
FROM facilities
WHERE facid IN (1, 5)
'''
pd.read_sql_query(query, con)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


## Question 5:
Produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100. Return the name and monthly maintenance of the facilities in question.

In [22]:
query = '''
SELECT name, monthlymaintenance, 
    CASE WHEN monthlymaintenance <= 100 THEN 'cheap'
        ELSE 'expensive' END AS cheap_or_expensive
FROM facilities
'''
pd.read_sql_query(query, con)

,name,monthlymaintenance,cheap_or_expensive
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


## Question 6:
You'd like to get the first and last name of the last member(s) who signed up. Try not to use the LIMIT clause for your solution.

In [39]:
query = '''
SELECT firstname, surname
FROM members
WHERE joindate IN 
    (SELECT MAX(joindate)
     FROM members)
'''
pd.read_sql_query(query, con)

,firstname,surname
0,Darren,Smith


## Question 7:
Produce a list of all members who have used a tennis court. Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name.

In [53]:
query = '''
SELECT DISTINCT m.firstname || ' ' || m.surname AS mem_name, f.name AS fac_name
FROM bookings AS b
    INNER JOIN members AS m ON b.memid=m.memid
    INNER JOIN facilities AS f ON b.facid=f.facid
WHERE f.name LIKE 'Tennis Court%'
    AND m.firstname <> 'GUEST'
ORDER BY m.surname, f.name
'''
pd.read_sql_query(query, con)

,mem_name,fac_name
0,Florence Bader,Tennis Court 1
1,Florence Bader,Tennis Court 2
2,Anne Baker,Tennis Court 1
3,Timothy Baker,Tennis Court 1
4,Anne Baker,Tennis Court 2
5,Timothy Baker,Tennis Court 2
6,Tim Boothe,Tennis Court 1
7,Tim Boothe,Tennis Court 2
8,Gerald Butters,Tennis Court 1
9,Gerald Butters,Tennis Court 2


## Question 8:
Produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30. Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.

In [81]:
query = '''
SELECT m.firstname || ' ' || m.surname AS mem_name,
    CASE WHEN b.memid=0 THEN SUM(b.slots*f.guestcost)
         ELSE SUM(b.slots*f.membercost) END AS total_cost
FROM bookings AS b
    INNER JOIN facilities AS f ON b.facid=f.facid
    INNER JOIN members AS m ON b.memid=m.memid
WHERE starttime LIKE '2012-09-14%'
GROUP BY mem_name
HAVING total_cost > 0
ORDER BY total_cost DESC
'''
pd.read_sql_query(query, con)

,mem_name,total_cost
0,GUEST GUEST,1315.0
1,Jemima Farrell,59.4
2,Burton Tracy,34.8
3,Tim Boothe,30.0
4,David Jones,30.0
5,David Pinker,22.0
6,Ponder Stibbons,19.8
7,Matthew Genting,19.8
8,Jack Smith,19.8
9,Florence Bader,19.8


## Question 9:
This time, produce the same result as in Q8, but using a subquery.

In [80]:
query = '''
WITH cost_table AS (
SELECT m.firstname || ' ' || m.surname AS mem_name, b.memid, b.facid,
    CASE WHEN b.memid=0 THEN b.slots*f.guestcost
         ELSE b.slots*f.membercost END AS booking_cost
FROM bookings AS b
    INNER JOIN members AS m USING(memid)
    INNER JOIN facilities AS f USING(facid)
WHERE starttime LIKE '2012-09-14%'
)
SELECT mem_name, SUM(booking_cost) AS total_cost
FROM cost_table
GROUP BY mem_name
HAVING total_cost > 0
ORDER BY total_cost DESC
'''
pd.read_sql_query(query, con)

,mem_name,total_cost
0,GUEST GUEST,1315.0
1,Jemima Farrell,59.4
2,Burton Tracy,34.8
3,Tim Boothe,30.0
4,David Jones,30.0
5,David Pinker,22.0
6,Ponder Stibbons,19.8
7,Matthew Genting,19.8
8,Jack Smith,19.8
9,Florence Bader,19.8


## Question 10:
Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [87]:
query = '''
WITH rev_table AS (
SELECT b.facid, f.name,
    CASE WHEN b.memid=0 THEN b.slots*f.guestcost
         ELSE b.slots*f.membercost END AS revenue
FROM bookings AS b
    INNER JOIN members AS m USING(memid)
    INNER JOIN facilities AS f USING(facid)
)
SELECT name AS facility_name, SUM(revenue) AS total_revenue
FROM rev_table
GROUP BY facility_name
ORDER BY total_revenue
'''
pd.read_sql_query(query, con)

,facility_name,total_revenue
0,Table Tennis,180.0
1,Snooker Table,240.0
2,Pool Table,270.0
3,Badminton Court,1906.5
4,Squash Court,13468.0
5,Tennis Court 1,13860.0
6,Tennis Court 2,14310.0
7,Massage Room 2,14454.6
8,Massage Room 1,50351.6


## Question 11:
Produce a report of members and who recommended them in alphabetic surname,firstname order.

In [94]:
query = '''
SELECT m1.firstname || ' ' || m1.surname AS member_name, m2.firstname || ' ' || m2.surname AS recommender_name
FROM members AS m1
    INNER JOIN members AS m2 ON m1.recommendedby=m2.memid
WHERE m1.recommendedby IS NOT NULL
ORDER BY m1.surname, m1.firstname
'''
pd.read_sql_query(query, con)

,member_name,recommender_name
0,Florence Bader,Ponder Stibbons
1,Anne Baker,Ponder Stibbons
2,Timothy Baker,Jemima Farrell
3,Tim Boothe,Tim Rownam
4,Gerald Butters,Darren Smith
5,Joan Coplin,Timothy Baker
6,Erica Crumpet,Tracy Smith
7,Nancy Dare,Janice Joplette
8,Matthew Genting,Gerald Butters
9,John Hunt,Millicent Purview


## Question 12:
Find the facilities with their usage by member, but not guests

In [102]:
query = '''
SELECT f.name, SUM(slots * 0.5) AS mem_usage_hrs
FROM facilities AS f
    INNER JOIN bookings AS b USING(facid)
WHERE memid <> 0
GROUP BY f.name
'''
pd.read_sql_query(query, con)

,name,mem_usage_hrs
0,Badminton Court,543.0
1,Massage Room 1,442.0
2,Massage Room 2,27.0
3,Pool Table,428.0
4,Snooker Table,430.0
5,Squash Court,209.0
6,Table Tennis,397.0
7,Tennis Court 1,478.5
8,Tennis Court 2,441.0


## Question 13:
Find the facilities usage by month, but not guests

In [106]:
query = '''
SELECT DISTINCT SUBSTRING(starttime, 6, 2) AS month, f.name, SUM(0.5*slots) AS mem_usage_hrs
FROM bookings AS b
    INNER JOIN facilities AS f USING(facid)
GROUP BY month, name
ORDER BY month, name
'''
pd.read_sql_query(query, con)

,month,name,mem_usage_hrs
0,07,Badminton Court,90.0
1,07,Massage Room 1,132.0
2,07,Massage Room 2,12.0
3,07,Pool Table,58.5
4,07,Snooker Table,78.0
5,07,Squash Court,82.0
6,07,Table Tennis,52.0
7,07,Tennis Court 1,135.0
8,07,Tennis Court 2,103.5
9,08,Badminton Court,229.5
